In [1]:
import numpy as np
import pandas as pd

In [2]:
objects = ['ecoli', 'scerevisiae', 'hela']

In [3]:
proteomes = {i: pd.read_excel(f'../data/processed/proteomes/{i}_proteome.xlsx') for i in objects}

In [4]:
datasets_list = pd.read_excel('../data/custom/studies_internal.xlsx')

In [5]:
datasets_ecoli = {i: pd.read_excel(f'../data/prepared/ecoli/{i}_int.xlsx') for i in datasets_list.loc[datasets_list['object'] == 'ecoli', 'study'].to_list()}

for i in datasets_ecoli:
    datasets_ecoli[i].iloc[:, -1] = datasets_ecoli[i].iloc[:, -1].fillna(0)
    datasets_ecoli[i] = datasets_ecoli[i][datasets_ecoli[i].iloc[:, -1] != 0.0] 
    
datasets_ecoli['soufi2015']['b_number'] = datasets_ecoli['soufi2015']['gene_names'].str.extract(r'(b\d{4})')
datasets_ecoli['soufi2015'] = datasets_ecoli['soufi2015'][['gene_names', 'b_number', 'copies_per_cell']]

In [6]:
proteomes['ecoli'] = proteomes['ecoli'][proteomes['ecoli']['is_reviewed'] == 'reviewed']
proteomes['ecoli']['gene_name'] = proteomes['ecoli']['gene_name'].str.lower()

In [7]:
for i in datasets_ecoli:
    datasets_ecoli[i]['is_group'] = False
    
    datasets_ecoli[i]['assigned_id'] = np.nan
    datasets_ecoli[i]['assigned_id_origin'] = np.nan
    
    if 'uniprot_id' in datasets_ecoli[i].columns:
        correct_ids = datasets_ecoli[i]['uniprot_id'].isin(proteomes['ecoli']['uniprot_id'])
        
        datasets_ecoli[i].loc[correct_ids, 'assigned_id'] = datasets_ecoli[i]['uniprot_id']
        datasets_ecoli[i].loc[correct_ids, 'assigned_id_origin'] = 'uniprot_id'
    
    if 'b_number' in datasets_ecoli[i].columns:
        ids_from_b_number = datasets_ecoli[i]['b_number'].map(proteomes['ecoli'].drop_duplicates('b_number').set_index('b_number')['uniprot_id'])
        
        datasets_ecoli[i]['assigned_id'] = datasets_ecoli[i]['assigned_id'].combine_first(ids_from_b_number)
        datasets_ecoli[i]['assigned_id_origin'] = datasets_ecoli[i]['assigned_id_origin'].combine_first(pd.Series(np.repeat('b_number', len(datasets_ecoli[i]))))
        
    if 'gene_name' in datasets_ecoli[i].columns:
        ids_from_gene_name = datasets_ecoli[i]['gene_name'].str.lower().map(proteomes['ecoli'].set_index('gene_name')['uniprot_id'])
        
        datasets_ecoli[i]['assigned_id'] = datasets_ecoli[i]['assigned_id'].combine_first(ids_from_gene_name)
        datasets_ecoli[i]['assigned_id_origin'] = datasets_ecoli[i]['assigned_id_origin'].combine_first(pd.Series(np.repeat('gene_name', len(datasets_ecoli[i]))))
    
    if 'uniprot_id' in datasets_ecoli[i].columns:
        datasets_ecoli[i]['is_group'] = datasets_ecoli[i]['uniprot_id'].str.contains('[\;\ \^\_]')
        datasets_ecoli[i].loc[datasets_ecoli[i]['is_group'], 'assigned_id'] = np.nan
        datasets_ecoli[i].loc[datasets_ecoli[i]['is_group'], 'assigned_id_origin'] = np.nan   

In [8]:
datasets_scerevisiae = {i: pd.read_excel(f'../data/prepared/scerevisiae/{i}_int.xlsx') for i in datasets_list.loc[datasets_list['object'] == 'scerevisiae', 'study'].to_list()}

for i in datasets_scerevisiae:
    datasets_scerevisiae[i].iloc[:, -1] = datasets_scerevisiae[i].iloc[:, -1].fillna(0)
    datasets_scerevisiae[i] = datasets_scerevisiae[i][datasets_scerevisiae[i].iloc[:, -1] != 0.0] 

datasets_scerevisiae['lahtvee2017']['copies_per_cell'] = datasets_scerevisiae['lahtvee2017']['copies_per_pgdw'] * 13

datasets_scerevisiae['lawless2016']['copies_per_cell'] = datasets_scerevisiae['lawless2016']['copies_per_cell'].astype(str)
datasets_scerevisiae['lawless2016'] = datasets_scerevisiae['lawless2016'][~datasets_scerevisiae['lawless2016']['copies_per_cell'].str.contains('-')]

datasets_scerevisiae['lawless2016_max'] = datasets_scerevisiae['lawless2016'].copy()
datasets_scerevisiae['lawless2016_max']['copies_per_cell'] = datasets_scerevisiae['lawless2016_max']['copies_per_cell'].str.replace('<', '')
datasets_scerevisiae['lawless2016_max']['copies_per_cell'] = datasets_scerevisiae['lawless2016_max']['copies_per_cell'].astype(float)

datasets_scerevisiae['lawless2016'] = datasets_scerevisiae['lawless2016'][~datasets_scerevisiae['lawless2016']['copies_per_cell'].str.contains('<')]
datasets_scerevisiae['lawless2016']['copies_per_cell'] = datasets_scerevisiae['lawless2016']['copies_per_cell'].astype(float)

In [9]:
proteomes['scerevisiae'] = proteomes['scerevisiae'][proteomes['scerevisiae']['is_reviewed'] == 'reviewed']

In [10]:
for i in datasets_scerevisiae:
    datasets_scerevisiae[i]['is_group'] = False
    
    datasets_scerevisiae[i]['assigned_id'] = np.nan
    datasets_scerevisiae[i]['assigned_id_origin'] = np.nan
    
    if 'uniprot_id' in datasets_scerevisiae[i].columns:
        correct_ids = datasets_scerevisiae[i]['uniprot_id'].isin(proteomes['scerevisiae']['uniprot_id'])
        
        datasets_scerevisiae[i].loc[correct_ids, 'assigned_id'] = datasets_scerevisiae[i]['uniprot_id']
        datasets_scerevisiae[i].loc[correct_ids, 'assigned_id_origin'] = 'uniprot_id'
    
    if 'orf_id' in datasets_scerevisiae[i].columns:
        ids_from_orf_id = datasets_scerevisiae[i]['orf_id'].map(proteomes['scerevisiae'].set_index('orf_id')['uniprot_id'])
        
        datasets_scerevisiae[i]['assigned_id'] = datasets_scerevisiae[i]['assigned_id'].combine_first(ids_from_orf_id)
        datasets_scerevisiae[i]['assigned_id_origin'] = datasets_scerevisiae[i]['assigned_id_origin'].combine_first(pd.Series(np.repeat('orf_id', len(datasets_scerevisiae[i]))))
        
    if 'gene_name' in datasets_scerevisiae[i].columns:
        ids_from_gene_name = datasets_scerevisiae[i]['gene_name'].str.lower().map(proteomes['scerevisiae'].drop_duplicates('gene_name').set_index('gene_name')['uniprot_id'])
        
        datasets_scerevisiae[i]['assigned_id'] = datasets_scerevisiae[i]['assigned_id'].combine_first(ids_from_gene_name)
        datasets_scerevisiae[i]['assigned_id_origin'] = datasets_scerevisiae[i]['assigned_id_origin'].combine_first(pd.Series(np.repeat('gene_name', len(datasets_scerevisiae[i]))))
    
    if 'uniprot_id' in datasets_scerevisiae[i].columns:
        datasets_scerevisiae[i]['is_group'] = datasets_scerevisiae[i]['uniprot_id'].str.contains('[\;\ \^\_]')
        datasets_scerevisiae[i].loc[datasets_scerevisiae[i]['is_group'], 'assigned_id'] = np.nan
        datasets_scerevisiae[i].loc[datasets_scerevisiae[i]['is_group'], 'assigned_id_origin'] = np.nan
        
    elif 'orf_id' in datasets_scerevisiae[i].columns:
        datasets_scerevisiae[i]['is_group'] = datasets_scerevisiae[i]['orf_id'].str.contains('[\;\ \^\_]')
        datasets_scerevisiae[i].loc[datasets_scerevisiae[i]['is_group'], 'assigned_id'] = np.nan
        datasets_scerevisiae[i].loc[datasets_scerevisiae[i]['is_group'], 'assigned_id_origin'] = np.nan

In [11]:
datasets_hela = {i: pd.read_excel(f'../data/prepared/hela/{i}_int.xlsx') for i in datasets_list.loc[datasets_list['object'] == 'hela', 'study'].to_list()}

for i in datasets_hela:
    datasets_hela[i].iloc[:, -1] = datasets_hela[i].iloc[:, -1].fillna(0)
    datasets_hela[i] = datasets_hela[i][datasets_hela[i].iloc[:, -1] != 0.0] 
    
datasets_hela['wisniewski2012']['copies_per_cell'] = datasets_hela['wisniewski2012']['copies_per_cell'].astype(str)

datasets_hela['wisniewski2012_max'] = datasets_hela['wisniewski2012'].copy()
datasets_hela['wisniewski2012_max']['copies_per_cell'] = datasets_hela['wisniewski2012_max']['copies_per_cell'].str.replace('<', '')
datasets_hela['wisniewski2012_max']['copies_per_cell'] = datasets_hela['wisniewski2012_max']['copies_per_cell'].astype(float)

datasets_hela['wisniewski2012'] = datasets_hela['wisniewski2012'][~datasets_hela['wisniewski2012']['copies_per_cell'].str.contains('<')]
datasets_hela['wisniewski2012']['copies_per_cell'] = datasets_hela['wisniewski2012']['copies_per_cell'].astype(float)

In [12]:
proteomes['hela'] = proteomes['hela'][proteomes['hela']['is_reviewed'] == 'reviewed']

In [13]:
def only_reviewed_ids(x):
    if ';' in x:
        x = pd.Series(x.split(';'))
        x = x[x.isin(proteomes['hela']['uniprot_id'])]
        x = ';'.join(x.values)
    return x

In [14]:
for i in datasets_hela:
    datasets_hela[i]['reviewed_id'] = datasets_hela[i]['uniprot_id'].apply(only_reviewed_ids)
    
    datasets_hela[i]['is_group'] = False
    
    datasets_hela[i]['assigned_id'] = np.nan
    datasets_hela[i]['assigned_id_origin'] = np.nan
    
    correct_ids = datasets_hela[i]['reviewed_id'].isin(proteomes['hela']['uniprot_id'])
        
    datasets_hela[i].loc[correct_ids, 'assigned_id'] = datasets_hela[i]['reviewed_id']
    datasets_hela[i].loc[correct_ids, 'assigned_id_origin'] = 'reviewed_id'
    
    datasets_hela[i]['is_group'] = datasets_hela[i]['reviewed_id'].str.contains('[\;\ \^\_]')
    datasets_hela[i].loc[datasets_hela[i]['is_group'], 'assigned_id'] = np.nan
    datasets_hela[i].loc[datasets_hela[i]['is_group'], 'assigned_id_origin'] = np.nan
    
    datasets_hela[i]['reviewed_id'] = datasets_hela[i]['reviewed_id'].replace('', np.nan)

In [15]:
for i in datasets_ecoli:
    datasets_ecoli[i].to_excel(f'../data/interim/ecoli/{i}_int.xlsx', index=False)
    datasets_ecoli[i] = datasets_ecoli[i][['assigned_id', 'copies_per_cell']]
    
    other = datasets_ecoli[i][datasets_ecoli[i]['assigned_id'].isna()]['copies_per_cell'].sum()
    datasets_ecoli[i] = datasets_ecoli[i][~datasets_ecoli[i]['assigned_id'].isna()]
    datasets_ecoli[i].loc[len(datasets_ecoli[i])] = ['_other', other]
    
    datasets_ecoli[i] = datasets_ecoli[i].groupby('assigned_id').sum().reset_index()
    
    datasets_ecoli[i].to_excel(f'../data/processed/ecoli/{i}_processed.xlsx', index=False)
    
for i in datasets_scerevisiae:
    datasets_scerevisiae[i].to_excel(f'../data/interim/scerevisiae/{i}_int.xlsx', index=False)
    datasets_scerevisiae[i] = datasets_scerevisiae[i][['assigned_id', 'copies_per_cell']]
    
    other = datasets_scerevisiae[i][datasets_scerevisiae[i]['assigned_id'].isna()]['copies_per_cell'].sum()
    datasets_scerevisiae[i] = datasets_scerevisiae[i][~datasets_scerevisiae[i]['assigned_id'].isna()]
    datasets_scerevisiae[i].loc[len(datasets_scerevisiae[i])] = ['_other', other]
    
    datasets_scerevisiae[i] = datasets_scerevisiae[i].groupby('assigned_id').sum().reset_index()
    
    datasets_scerevisiae[i].to_excel(f'../data/processed/scerevisiae/{i}_processed.xlsx', index=False)
    
for i in datasets_hela:
    datasets_hela[i].to_excel(f'../data/interim/hela/{i}_int.xlsx', index=False)
    datasets_hela[i] = datasets_hela[i][['assigned_id', 'copies_per_cell']]
    
    other = datasets_hela[i][datasets_hela[i]['assigned_id'].isna()]['copies_per_cell'].sum()
    datasets_hela[i] = datasets_hela[i][~datasets_hela[i]['assigned_id'].isna()]
    datasets_hela[i].loc[len(datasets_hela[i])] = ['_other', other]
    
    datasets_hela[i] = datasets_hela[i].groupby('assigned_id').sum().reset_index()
    
    datasets_hela[i].to_excel(f'../data/processed/hela/{i}_processed.xlsx', index=False)

In [16]:
datasets_list.loc[len(datasets_list)] = ['scerevisiae', 'lawless2016_max', 'LA16', 'MS']
datasets_list.loc[len(datasets_list)] = ['hela', 'wisniewski2012_max', 'WI12', 'MS']

In [17]:
datasets_list.to_excel('../data/processed/datasets_list.xlsx', index=False)